# Dependancies

## Requirements

In [1]:
#!pip install sentence_transformers langchain openai tqdm datasets asyncio scikit-learn cohere tiktoken umap altair

In [2]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

import openai
import instructor

from src.bubble import *
from src.models import *
from src.utilities import *


Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Darty_trustpilot : 1707329196900x870734705097005300


In [3]:
#PROJECT =  "Metro" #"Cheerz"
#project_path = 'Results/'+PROJECT
#os.makedirs(project_path, exist_ok=True)

In [4]:
aspects_df = get("Aspect")

In [5]:
aspects_df.head()

,Company,Project,Rating,SubCategory,Associated_feedback,Date,Category,_id,Explanation
0,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1709253069790x305647624075015900,1707643040311x281208184060071520,2024-02-07 00:00:00+00:00,1709253069102x717430988942903300,1709288669731x792381451006279400,NaN
1,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1709253063096x594485916638931600,1707643040311x281208184060071520,2024-02-07 00:00:00+00:00,1709253062423x867774813720628500,1709288669735x628249787027423700,NaN
2,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1709253066544x770991303066428400,1707643040311x281208184060071520,2024-02-07 00:00:00+00:00,1709253065849x444427432726514300,1709288669739x307796151670405300,La touche Touch ID permet de s'authentifier ra...
3,1707313014508x102198350946437700,1707329196900x870734705097005300,3,1709253068464x983179727881815700,1707643040311x281208184060071520,2024-02-07 00:00:00+00:00,1709253065849x444427432726514300,1709288669741x932504918890495000,"Le prix est élevé, comme pour tous les produit..."
4,1707313014508x102198350946437700,1707329196900x870734705097005300,5,1709253077078x822788218820358100,1707643040316x844188021545895200,2024-01-31 00:00:00+00:00,1709253075821x830225984700473300,1709288671136x151764047078959840,La prise en charge de l'installation de la mac...


# Insights extraction

In [6]:
TYPES_LIST = ['Point positif', 'Nouvelle fonctionnalité', 'Point de douleur', 'Bug']

tags_df = get("Tag", constraints=[])
categories_df = get("Category")
subcategories_df = get("SubCategory")

In [7]:
company_infos = bubble_client.get(
    "Company",
    bubble_id=COMPANY_ID,
)
project_infos = bubble_client.get(
    "Project",
    bubble_id=PROJECT_ID,
)

feedback_context = {
    "entreprise": company_infos["Name"],
    "context": company_infos['Context'],
    "role": company_infos['Role'],
    "cible": project_infos['Target'],
    "types": '- '+' \n- '.join(TYPES_LIST),
    "tags": '- '+' \n- '.join([row["Name"]+' : '+row["Description"] for _,row in tags_df.iterrows()]),
    #"insight_types": types_descr,
    #"insight_categories": tags_descr,
    #"question": project_infos['Study_question'],
    #"exemple_commentaire": exemple_commentaire,
    #"example_insights": '\n- '.join(list(examples_insights_df['Insights qui devraient en découler'])),
}

feedback_context

{'entreprise': 'Darty',
 'context': 'Fondée en 1957, Darty est une enseigne française spécialisée dans la distribution d\'électroménager, d\'équipements électroniques et de produits culturels. Rachetée par la Fnac en 2016, elle est aujourd\'hui l\'un des leaders européens de la distribution omnicanale.\n\nÉvènements récents:\n\n    2016: Rachat par la Fnac et création du groupe Fnac Darty.\n    2017: Lancement de la marketplace Darty.com.\n    2018: Déploiement du "Contrat de Confiance Fnac Darty" dans tous les magasins.\n    2019: Lancement de l\'offre de services "Darty+."\n    2020: Accélération de la transformation digitale du groupe.\n    2021: Acquisition de Mistergooddeal, spécialiste du e-commerce en produits reconditionnés.\n    2022: Lancement de la Fnac Darty Academy, une plateforme de formation en ligne.\n\nConcurrents:\n\n    Boulanger\n    Conforama\n    Gitem\n    Amazon\n    Cdiscount\n\nEnjeux:\n\n    Darty doit faire face à une concurrence accrue sur le marché de l\'é

In [8]:
ID_CATEG_NONE = categories_df[categories_df["Name"].isna()].iloc[0]["_id"]
SUBCATEG_NONE = subcategories_df[subcategories_df["Name"].isna()]
ID_CATEG_NONE, SUBCATEG_NONE

('1709322143530x849396050152903400',
                              Company Name                           Project  \
 24  1707313014508x102198350946437700  NaN  1707329196900x870734705097005300   
 25  1707313014508x102198350946437700  NaN  1707329196900x870734705097005300   
 26  1707313014508x102198350946437700  NaN  1707329196900x870734705097005300   
 27  1707313014508x102198350946437700  NaN  1707329196900x870734705097005300   
 28  1707313014508x102198350946437700  NaN  1707329196900x870734705097005300   
 29  1707313014508x102198350946437700  NaN  1707329196900x870734705097005300   
 30  1707313014508x102198350946437700  NaN  1707329196900x870734705097005300   
 
                             Category                               _id  
 24  1709322143530x849396050152903400  1709322250500x203169341028516480  
 25  1709253062423x867774813720628500  1709328814420x450890791939818430  
 26  1709253065849x444427432726514300  1709328848587x188405163211254430  
 27  1709253069102x717430

In [9]:
TypeInsight = enum.Enum("Type de l'insight", [(convert_text_to_constants(t), t) for t in TYPES_LIST])
#types_to_id = {convert_text_to_constants(row.Name): row._id for _, row in types_df.iterrows()}

TagInsight = enum.Enum("Tag de l'insight", [(convert_text_to_constants(row.Name), row.Name) for _, row in tags_df.iterrows()])
tags_to_id = {convert_text_to_constants(row.Name): row._id for _, row in tags_df.iterrows()}


In [47]:
#FeedbackIndex = enum.Enum("Indice du retour associé", [(str(i), i) for i in range(BATCH_SIZE)])

class Insight(BaseModel):
    associated_indexes: List[int] = Field(description="Indices des retours associés.")
    insight: str = Field(description="Insight qui a été déduit.") 
    type_insight: TypeInsight = Field(description="Type de l'insight.") 
    def __str__(self):
        return self.insight + '(tag: '+self.type_insight+') (indexes: ' + str(self.associated_indexes)+')'

class Consequence(BaseModel):
    title: str = Field(description="Titre de la conséquence")
    detail: str = Field(description="Phrase décrivant la conséquence.")
    def __str__(self):
        return self.title +' : ' + self.detail

class SubCategory(BaseModel):
    #model_config = ConfigDict(title='Main')
    
    #insight_categories: List[str] = Field(description="Categories de l'insight.")
    #insight_type: TypeInsight = Field(description="Type de l'insight.")
    #insight_tags: List[TagInsight] = Field(description="Tags de l'insight.")

    title: str = Field(description="Titre de cette sous-catégorie.")
    insights: List[Insight] = Field(description="Liste des insights qui ont été déduits.") #Field(description="Point intéressant a retenir du commentaire.")
    #negative_insights: List[Insight] = Field(description="Liste des insights négatifs qui ont été déduits.") #Field(description="Point intéressant a retenir du commentaire.")
    consequences: List[Consequence] = Field(description="Conséquences pour l'entrerpise.") #Field(description="Point intéressant a retenir du commentaire.")
    #suggestion: str = Field(description="Suggestion faite à l'enteprise pour l'aider a gérer la situation.") #Field(description="Point intéressant a retenir du commentaire.")
    #category = ""
    #sub_category = ""

    def __str__(self):
        return self.title + "\n\n- " + '\n- '.join([str(x) for x in self.insight]) + "\n\n- " + '\n- '.join([str(x) for x in self.consequences])
    
class ListSubCategory(BaseModel):
    sub_categories_list: List[SubCategory] = Field(description="Liste des insights.")

#FirstInsight.model_json_schema() 
ListSubCategory(sub_categories_list=[SubCategory(title='Accueil et comportement du personnel', insights=[Insight(associated_indexes=[20, 96, 123, 475, 891, 1249, 1267, 1372, 1965], insight='Accueil chaleureux, écoute attentive et conseils pertinents', type_insight=TypeInsight.BUG)], consequences=[])])

ListSubCategory(sub_categories_list=[SubCategory(title='Accueil et comportement du personnel', insights=[Insight(associated_indexes=[20, 96, 123, 475, 891, 1249, 1267, 1372, 1965], insight='Accueil chaleureux, écoute attentive et conseils pertinents', type_insight=<Type de l'insight.BUG: 'Bug'>)], consequences=[])])

In [11]:
categories_df

,Company,Name,Project,_id
0,1707313014508x102198350946437700,Service Client,1707329196900x870734705097005300,1709253062423x867774813720628500
1,1707313014508x102198350946437700,Produits,1707329196900x870734705097005300,1709253065849x444427432726514300
2,1707313014508x102198350946437700,Expérience en Magasin,1707329196900x870734705097005300,1709253069102x717430988942903300
3,1707313014508x102198350946437700,Site Web et Application,1707329196900x870734705097005300,1709253072455x374665745287587650
4,1707313014508x102198350946437700,Livraison et Installation,1707329196900x870734705097005300,1709253075821x830225984700473300
5,1707313014508x102198350946437700,Politique de Retour,1707329196900x870734705097005300,1709253079053x541586600770979300
6,1707313014508x102198350946437700,NaN,1707329196900x870734705097005300,1709322143530x849396050152903400


In [12]:
with open('Prompts/fr/prompt_regroupement.txt') as f:
    prompt_regroupement = PromptTemplate.from_template(f.read())


print(prompt_regroupement.template)

Tu est un consultant extrèmement intelligent qui travaille pour l'entreprise {entreprise}

Tu as mené une enquête auprès des {cible} de cette entreprise concernant la catégorie "{category}", et en a déduit plusieurs choses.
Ton objectif est d'aider l'entreprise à améliorer son experience utilisateur et adapter sa stratégie. 
Tu doit choisir les retours intéressants, et les regrouper en insights distinctes.

Voici les étapes de l'analyse:

Étape 1
Regroupe les retours en sous-catégories de la catégorie "{category}".
Essaie de faire des regroupement de taille homogène.

Par exemple, au sein de la catégorie "Livraison et Installation : Fiabilité", de bons insights pourraient être: 
- Retards de livraison
- Livraison de produits endommagés
- Informations contradictoires du SAV
- Commandes annulées
alors que "Expériences positives" ne serait pas une sous-catégorie valide. 

Étape 2
Associe a chaque catégorie des insights positifs et négatifs. 
Un insight est un point intéressant de faire re

In [13]:
prompts = []
subcat_ids = []
for subcat_id, df in aspects_df[aspects_df['Explanation'].notna()].groupby('SubCategory'):
    subcat = subcategories_df[subcategories_df['_id'] == subcat_id].iloc[0]
    cat = categories_df[categories_df['_id'] == subcat['Category']].iloc[0]

    feedbacks = '\n'.join([str(index)+' : '+content for (index, content) in df['Explanation'].items()])
    
    prompts.append(prompt_regroupement.invoke({"feedbacks": feedbacks, "category": cat["Name"]+" : "+subcat['Name'], **feedback_context}).text)
    subcat_ids.append(subcat_id)

#print(prompts[0])
print("Traitement synchronisé de", len(prompts), "prompts.")
list_insights = apply_async_analysis(prompts, ListSubCategory)

Traitement synchronisé de 22 prompts.


Retrying, exception: 3 validation errors for ListSubCategory
sub_categories_list.0.consequence
  Field required [type=missing, input_value={'title': 'Processus de r...sus de remboursement'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
sub_categories_list.1.consequence
  Field required [type=missing, input_value={'title': 'Manque de tran...ons de remboursement'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
sub_categories_list.2.consequence
  Field required [type=missing, input_value={'title': "Comparaisons d... chez Amazon ou FNAC'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing
Traceback (most recent call last):
  File "/Users/gardille/opt/anaconda3/envs/feedback_env/lib/python3.11/site-packages/instructor/patch.py", line 246, in retry_async
    return await process_response_async(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/

In [18]:
list_insights[0].sub_categories_list[0]

SubCategory(title='Accueil et comportement du personnel', positiv_insights=[Insight(associated_indexes=[20, 96, 123, 475, 891, 1249, 1267, 1372, 1965], insight='Accueil chaleureux, écoute attentive et conseils pertinents'), Insight(associated_indexes=[901], insight='Grande satisfaction envers le personnel et le directeur du magasin')], negativ_insights=[Insight(associated_indexes=[53, 367, 420, 688, 897, 1635, 1636, 2016, 2139, 2143, 2153, 2161], insight='Comportement irrespectueux, moqueries et attitude agressive'), Insight(associated_indexes=[199, 300, 314, 491, 545, 674, 865, 1099], insight="Manque d'amabilité, d'accueil et de disponibilité")], consequence=[Consequence(title='Image de marque dégradée', detail="Le comportement irrespectueux et le manque de professionnalisme de certains membres du personnel nuisent à l'image de Darty."), Consequence(title='Perte de clients', detail="L'inefficacité du service client et le manque de communication conduisent à la perte de clients méconte

In [39]:
list_insights

[ListSubCategory(sub_categories_list=[SubCategory(title='Accueil et comportement du personnel', positiv_insights=[Insight(associated_indexes=[20, 96, 123, 475, 891, 1249, 1267, 1372, 1965], insight='Accueil chaleureux, écoute attentive et conseils pertinents'), Insight(associated_indexes=[901], insight='Grande satisfaction envers le personnel et le directeur du magasin')], negativ_insights=[Insight(associated_indexes=[53, 367, 420, 688, 897, 1635, 1636, 2016, 2139, 2143, 2153, 2161], insight='Comportement irrespectueux, moqueries et attitude agressive'), Insight(associated_indexes=[199, 300, 314, 491, 545, 674, 865, 1099], insight="Manque d'amabilité, d'accueil et de disponibilité")], consequence=[Consequence(title='Image de marque dégradée', detail="Le comportement irrespectueux et le manque de professionnalisme de certains membres du personnel nuisent à l'image de Darty."), Consequence(title='Perte de clients', detail="L'inefficacité du service client et le manque de communication co

In [14]:
print(prompts[0])


Tu est un consultant extrèmement intelligent qui travaille pour l'entreprise Darty

Tu as mené une enquête auprès des client de cette entreprise concernant la catégorie "Service Client : Accueil et assistance", et en a déduit plusieurs choses.
Ton objectif est d'aider l'entreprise à améliorer son experience utilisateur et adapter sa stratégie. 
Tu doit choisir les retours intéressants, et les regrouper en insights distinctes.

Voici les étapes de l'analyse:

Étape 1
Regroupe les retours en sous-catégories de la catégorie "Service Client : Accueil et assistance".
Essaie de faire des regroupement de taille homogène.

Par exemple, au sein de la catégorie "Livraison et Installation : Fiabilité", de bons insights pourraient être: 
- Retards de livraison
- Livraison de produits endommagés
- Informations contradictoires du SAV
- Commandes annulées
alors que "Expériences positives" ne serait pas une sous-catégorie valide. 

Étape 2
Associe a chaque catégorie des insights positifs et négatifs. 

In [40]:
def send_insights(insights_group, cat_id, subcat_id):

    if len(insights_group.positiv_insights)>0:
       
      res = bubble_client.create("Insight",
        [{
          "Company": COMPANY_ID,
          "Project": PROJECT_ID,
          "Name": insight.insight,
          "Category": cat_id,
          "SubCategory": subcat_id,
          "Type": insight.insight_type._value_,
          "Aspects": list(aspects_df.iloc[insight.associated_indexes]._id),
          "Feedbacks": list(aspects_df.iloc[insight.associated_indexes].Associated_feedback),
          "Nb Feedbacks": len(list(aspects_df.iloc[insight.associated_indexes].Associated_feedback)),
          }  for insight in insights_group.insights]                     
        )
      insights_id = [x['id'] for x in res]
    else:
      insights_id = []

    if len(insights_group.consequence)>0:
      res = bubble_client.create("Consequence",
        [{
          "Company": COMPANY_ID,
          "Project": PROJECT_ID,
          "Description": conseq.detail,
          "Name": conseq.title,
          }  for conseq in insights_group.consequences]                     
        )
      consequences_id = [x['id'] for x in res] 
    else:
      consequences_id = []


    bubble_id = bubble_client.create("Insights Group", {
      "Company": COMPANY_ID,
      "Project": PROJECT_ID,
      "Name": insights_group.title,
      "Category": cat_id,
      "SubCategory": subcat_id,
      "Insights": insights_id,
      "Consequences": consequences_id,
      #"Tags": [tags_to_id[tag._name_] for tag in insight.insight_tags],
      })

    

for (list_insight_groups, subcat_id) in tqdm(zip(list_insights, subcat_ids)):
  cat_id = subcategories_df[subcategories_df['_id'] == subcat_id].iloc[0].Category
  for insights_group in list_insight_groups.sub_categories_list:
    send_insights(insights_group, cat_id, subcat_id)

    empty_subcat = SUBCATEG_NONE[SUBCATEG_NONE["Category"] ==cat_id].iloc[0]._id
    send_insights(insights_group, cat_id, empty_subcat)

    empty_subcat = SUBCATEG_NONE[SUBCATEG_NONE["Category"] ==ID_CATEG_NONE].iloc[0]._id
    send_insights(insights_group, ID_CATEG_NONE, empty_subcat)


0it [00:00, ?it/s]

In [33]:
res = [{'status': 'success', 'id': '1709819458769x866368741440162400'}, {'status': 'success', 'id': '1709819458808x329560547820026100'}]


['1709819458769x866368741440162400', '1709819458808x329560547820026100']

In [24]:
insights_group

('sub_categories_list',
 [SubCategory(title='Accueil et comportement du personnel', positiv_insights=[Insight(associated_indexes=[20, 96, 123, 475, 891, 1249, 1267, 1372, 1965], insight='Accueil chaleureux, écoute attentive et conseils pertinents'), Insight(associated_indexes=[901], insight='Grande satisfaction envers le personnel et le directeur du magasin')], negativ_insights=[Insight(associated_indexes=[53, 367, 420, 688, 897, 1635, 1636, 2016, 2139, 2143, 2153, 2161], insight='Comportement irrespectueux, moqueries et attitude agressive'), Insight(associated_indexes=[199, 300, 314, 491, 545, 674, 865, 1099], insight="Manque d'amabilité, d'accueil et de disponibilité")], consequence=[Consequence(title='Image de marque dégradée', detail="Le comportement irrespectueux et le manque de professionnalisme de certains membres du personnel nuisent à l'image de Darty."), Consequence(title='Perte de clients', detail="L'inefficacité du service client et le manque de communication conduisent à l

In [25]:
list_insight_groups

ListSubCategory(sub_categories_list=[SubCategory(title='Accueil et comportement du personnel', positiv_insights=[Insight(associated_indexes=[20, 96, 123, 475, 891, 1249, 1267, 1372, 1965], insight='Accueil chaleureux, écoute attentive et conseils pertinents'), Insight(associated_indexes=[901], insight='Grande satisfaction envers le personnel et le directeur du magasin')], negativ_insights=[Insight(associated_indexes=[53, 367, 420, 688, 897, 1635, 1636, 2016, 2139, 2143, 2153, 2161], insight='Comportement irrespectueux, moqueries et attitude agressive'), Insight(associated_indexes=[199, 300, 314, 491, 545, 674, 865, 1099], insight="Manque d'amabilité, d'accueil et de disponibilité")], consequence=[Consequence(title='Image de marque dégradée', detail="Le comportement irrespectueux et le manque de professionnalisme de certains membres du personnel nuisent à l'image de Darty."), Consequence(title='Perte de clients', detail="L'inefficacité du service client et le manque de communication con

In [14]:
def send_insights(list_insights_subcat, cat_id, subcat_id):
    dicts = [{
    "Company": COMPANY_ID,
    "Project": PROJECT_ID,
    "Category": cat_id,
    "SubCategory": subcat_id,
    "Consequence": insight.consequence,
    "Content": insight.insight,
    "Suggestion": insight.suggestion,
    #"Type": insight.insight_type._value_,
    #"Tags": [tags_to_id[tag._name_] for tag in insight.insight_tags],
    "Aspects": list(aspects_df.iloc[insight.associated_indexes]._id),
    "Feedbacks": list(aspects_df.iloc[insight.associated_indexes].Associated_feedback),
    "Nb Feedbacks": len(list(aspects_df.iloc[insight.associated_indexes].Associated_feedback)),
    "Title": insight.title,
    }  for insight in list_insights_subcat.sub_categories_list]

    bubble_id = bubble_client.create("Insight",dicts)

    d_without_categories = []
    d_without_subcategories = []

    for d in dicts :
      d_without_subcategory = d.copy()
      d_without_subcategory["SubCategory"] = SUBCATEG_NONE[SUBCATEG_NONE["Category"] ==cat_id].iloc[0]._id
      d_without_subcategories.append(d_without_subcategory)
      
      d_without_category = d.copy()
      d_without_category["Category"] = ID_CATEG_NONE
      d_without_category["SubCategory"] = SUBCATEG_NONE[SUBCATEG_NONE["Category"] ==ID_CATEG_NONE].iloc[0]._id
      d_without_categories.append(d_without_category)

    bubble_id = bubble_client.create("Insight",d_without_subcategory)
    bubble_id = bubble_client.create("Insight",d_without_category)
    

for (list_insights_subcat, subcat_id) in zip(list_insights, subcat_ids):
  cat_id = subcategories_df[subcategories_df['_id'] == subcat_id].iloc[0].Category

  send_insights(list_insights_subcat, cat_id, subcat_id)

In [ ]:
prompts = []
BATCH_SIZE = 10

for batch_df in batchify(feedbacks_df, size=BATCH_SIZE):
    context = deepcopy(feedback_context)
    context["feedbacks"] = '\n'.join([str(i)+" : "+x for i, x in zip(batch_df.index, batch_df["content"])])  
    #"- "+"\n- ".join(batch_df['content'])
    #context["insights"] = "- "+"\n- ".join(batch_df['content'])
    prompts.append(prompt_insights.invoke(context))


In [ ]:
print(prompts[1].text)

Tu es analyste marketing au sein de l'entreprise suivante:
METRO est une enseigne réservée aux professionnels des métiers de bouche (restaurateurs, traiteurs, bouchers, boulangers…) qui vend des produits alimentaires et non alimentaires adaptés à leur activité, en magasins (appelés entrepôts ou halles) et en livraison. 

Tu as mené une enquête auprès des client de l'entreprise, et a récupéré une liste de retours. 
Tu es chargé de faire remonter auprès de l'entreprise les conlusion de ton enquète, c'est à dire les insights que tu as déduit de l'analyse de ces retours.

Effectue les étapes suivantes:

Étape 1 - Identification des insights
Identifie les insights à faire remonter auprès de ton équipe.
Voici les contraintes que les insights doivent respecter:
- Une personne de ton équipe qui lit un insight doit pouvoir en comprendre le sens, sans qu'il y ait d'ambiguité.
- Un insight doit être aussi court que possible, tout en restant parfaitement compréhensible et pertinent.
- N'ajoute pas

In [ ]:
responses[0].sub_categories_list

[FirstInsight(insight_categories=[<Categories de l'insight.LIVRAISON: 'Livraison'>], insight_type=<Type de l'insight.POINT_DE_DOULEUR: 'Point de douleur'>, associated_feedbacks=[0], contenu="La limitation de certains produits à l'achat en carton lors de la livraison contrairement à l'achat à l'unité en magasin réduit la flexibilité pour les clients."),
 FirstInsight(insight_categories=[<Categories de l'insight.EXPERIENCE_D_ACHAT: "Expérience d'achat">, <Categories de l'insight.POLITIQUE_DE_PRIX: 'Politique de prix'>], insight_type=<Type de l'insight.POINT_DE_DOULEUR: 'Point de douleur'>, associated_feedbacks=[1], contenu="L'offre en ligne est perçue comme moins avantageuse que l'achat en magasin, avec une impossibilité de comparer les prix et un sentiment d'obligation de se rendre en magasin."),
 FirstInsight(insight_categories=[<Categories de l'insight.MAGASIN: 'Magasin'>], insight_type=<Type de l'insight.POINT_DE_DOULEUR: 'Point de douleur'>, associated_feedbacks=[2], contenu='Le ran

In [ ]:
responses = apply_async_analysis(prompts, InsightsList)
list_batch_insights_df = [pd.DataFrame(enum_to_str(response.sub_categories_list)) for response in responses]

print(len(list_batch_insights_df), "batch have been processed")

2 batch have been processed


In [ ]:
responses

[InsightsList(insights_list=[FirstInsight(insight_categories=[<Categories de l'insight.LIVRAISON: 'Livraison'>], insight_type=<Type de l'insight.POINT_DE_DOULEUR: 'Point de douleur'>, associated_feedbacks=[0], contenu="La limitation de certains produits à l'achat en carton lors de la livraison contrairement à l'achat à l'unité en magasin réduit la flexibilité pour les clients."), FirstInsight(insight_categories=[<Categories de l'insight.EXPERIENCE_D_ACHAT: "Expérience d'achat">, <Categories de l'insight.POLITIQUE_DE_PRIX: 'Politique de prix'>], insight_type=<Type de l'insight.POINT_DE_DOULEUR: 'Point de douleur'>, associated_feedbacks=[1], contenu="L'offre en ligne est perçue comme moins avantageuse que l'achat en magasin, avec une impossibilité de comparer les prix et un sentiment d'obligation de se rendre en magasin."), FirstInsight(insight_categories=[<Categories de l'insight.MAGASIN: 'Magasin'>], insight_type=<Type de l'insight.POINT_DE_DOULEUR: 'Point de douleur'>, associated_feed

In [ ]:
[len(df) for df in list_batch_insights_df]

[10, 5]

In [ ]:
list_batch_insights_df[0]

,insight_categories,insight_type,associated_feedbacks,contenu
0,[Livraison],Point de douleur,[None],La limitation de certains produits à l'achat e...
1,"[Expérience d'achat, Politique de prix]",Point de douleur,[None],L'offre en ligne est perçue comme moins avanta...
2,[Magasin],Point de douleur,[None],"Le rangement en magasin est jugé désordonné, r..."
3,[],Point positif,[None],Les opérations de déstockage chez d'autres fou...
4,[Expérience d'achat],Point de douleur,[None],L'arrêt de la distribution des promotions sous...
5,"[Qualité des produits, Politique de prix]",Point de douleur,[None],Les prix de certains produits chez Metro sont ...
6,[Qualité des produits],Point de douleur,[None],La disparition de l'offre d'entrées chaudes et...
7,[Service client],Point de douleur,[None],Un manque de support pour charger les achats e...
8,[Politique de prix],Point de douleur,[None],"Les prix sont jugés trop élevés, même avec un ..."
9,"[Livraison, Service client]",Point de douleur,[None],Le processus de livraison et la gestion des av...


In [ ]:
list(list_batch_insights_df[0]['contenu'])

["La limitation de certains produits à l'achat en carton lors de la livraison contrairement à l'achat à l'unité en magasin réduit la flexibilité pour les clients.",
 "L'offre en ligne est perçue comme moins avantageuse que l'achat en magasin, avec une impossibilité de comparer les prix et un sentiment d'obligation de se rendre en magasin.",
 'Le rangement en magasin est jugé désordonné, rendant difficile la recherche de produits.',
 "Les opérations de déstockage chez d'autres fournisseurs sont très appréciées.",
 "L'arrêt de la distribution des promotions sous forme papier ou digitale est regretté.",
 'Les prix de certains produits chez Metro sont jugés plus élevés que ceux de la grande distribution, même pour des produits identiques.',
 "La disparition de l'offre d'entrées chaudes et de propositions traiteurs pendant les fêtes de fin d'année est regrettée.",
 'Un manque de support pour charger les achats en grande quantité dans le véhicule du client est signalé.',
 'Les prix sont jugé

## Accociate newly created insights to feedbacks 

In [ ]:
with open('Blumana-prompts/prompt_feedbacks.txt') as f:
    prompt_feedbacks = PromptTemplate.from_template(f.read())

In [ ]:
class Sentiment(str, enum.Enum):
    POSITIF = "Positif"
    NEUTRE = "Neutre"
    NEGATIF = "Négatif"


In [ ]:
InsightsIndex = enum.Enum("Indice de l'insight associé", [(str(i), i) for i in range(BATCH_SIZE)])

class Feedback(BaseModel):
        insights_list: List[InsightsIndex] = Field(description="Indices des insights associés à ce retour")
        sentiment: Sentiment = Field(description="Sentiment exprimé")

class FeedbackInfosList(BaseModel):
        feedbacks_list: List[Feedback] = Field(description="Liste des informations associées aux feedbacks.")

In [ ]:
prompts = []
for batch_insights_df, batch_feedbacks_df in zip(list_batch_insights_df, batchify(feedbacks_df, size=BATCH_SIZE)):
    #InsightsEnum = enum.Enum("Insight associé", [(convert_text_to_constants(x), i) for i, x in zip(batch_insights_df.index, batch_insights_df["content"])])

    context = deepcopy(feedback_context)
    #context["feedbacks"] = "- "+"\n- ".join(batch_feedbacks_df['content'])
    context["feedbacks"] = '\n'.join([str(i)+" : "+x for i, x in zip(batch_insights_df.index, batch_feedbacks_df["content"])])  
    context["insights"] = '\n'.join([str(i)+" : "+x for i, x in zip(batch_insights_df.index, batch_insights_df["contenu"])])
    prompts.append(prompt_feedbacks.invoke(context))



In [ ]:
print(prompts[0].text)

Tu es analyste marketing au sein de l'entreprise suivante:
METRO est une enseigne réservée aux professionnels des métiers de bouche (restaurateurs, traiteurs, bouchers, boulangers…) qui vend des produits alimentaires et non alimentaires adaptés à leur activité, en magasins (appelés entrepôts ou halles) et en livraison. 

Tu as mené une enquête auprès des client de l'entreprise. 
Tu as récupérés des commentaires, et en a extrait des insights.
               
Pour chacun des retours qui te seront donnés, effectue les étapes suivantes:

Étape 1 - Identifie si le sentiment exprimé dans chacun des retours par le client est "Positif", "Neutre" ou "Négatif". Prends en compte la formulation de la question ayant été posée (Il s’agit d’une étude clients pour mesurer l’image prix de l’enseigne METRO. En fin de questionnaire, une question ouverte permet aux clients de laisser s’ils le souhaitent un commentaire libre. Ils peuvent exprimer leurs points de satisfaction et d’insatisfaction sur les pri

In [ ]:

responses = apply_async_analysis(prompts, FeedbackInfosList)

list_enriched_feedbacks_df = [pd.DataFrame(enum_to_str(response.feedbacks_list)) for response in responses]

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_68618/2334417953.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  return enum_to_str(e.dict())


In [ ]:
[len(df) for df in list_enriched_feedbacks_df]

[9, 5]

In [ ]:
len(pd.concat(list_enriched_feedbacks_df))

14

In [ ]:
for batch_insights_df, batch_index_feedbacks, enriched_feedbacks_df in zip(list_batch_insights_df, batchify(feedbacks_df.index, size=BATCH_SIZE), list_enriched_feedbacks_df):
    feedbacks_df.loc[batch_index_feedbacks, 'sentiment'] = enriched_feedbacks_df['sentiment']
    feedbacks_df.loc[batch_index_feedbacks, 'insights_index'] = enriched_feedbacks_df['insights_list']

In [ ]:
batch_insights_df

,insight_categories,insight_type,contenu
0,[Politique de prix],Point de douleur,"Les prix de certains produits (bananes, frambo..."
1,[Qualité des produits],Point de douleur,"La qualité et la fraîcheur des produits, notam..."
2,[Expérience d'achat],Point de douleur,Le nouveau directeur de Metro Caen est perçu c...
3,"[Disponibilité des produits, Livraison]",Point de douleur,La réduction de l'assortiment disponible à la ...
4,[Politique de prix],Point de douleur,Les promotions sont jugées insuffisantes pour ...
5,[],Nouvelle demande,Mise en place de points de fidélité suggérée.


In [ ]:
list_batch_feedbacks_df = [pd.DataFrame(enum_to_str(response.feedbacks_list)) for response in responses]
list_batch_feedbacks_df

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_68618/2334417953.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  return enum_to_str(e.dict())


[  insights_list sentiment
 0            []   Positif
 1           [0]   Négatif
 2           [1]   Négatif
 3           [2]   Positif
 4           [3]   Négatif
 5           [4]   Négatif
 6           [5]   Négatif
 7           [6]   Négatif
 8           [7]   Négatif,
   insights_list sentiment
 0           [4]   Négatif
 1           [5]   Positif
 2           [3]   Négatif
 3        [0, 1]   Négatif
 4           [2]   Négatif]

In [ ]:
feedbacks_df

,Modified Date,Created Date,Created By,content,company,sentiment,Analyzed?,source,character_number,insights,_id,insights_index
0,2024-01-31 15:19:21.203000+00:00,2024-01-21 15:40:00.025000+00:00,1705847494855x437900943146650500,livrer TOUT les produits disponibles en magasi...,1705585399217x205117684451615600,Positif,False,1705851599107x404539534708310000,95,"[1706714349371x275598239568271680, 17067143467...",1705851599759x115801332943705310,[]
1,2024-01-31 15:19:21.577000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,Votre offer internet devient « ridicule ». Ell...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,486,"[1706714348320x182894283191188160, 17067143465...",1705851599759x118530353766926000,[0]
2,2024-01-31 15:19:21.952000+00:00,2024-01-21 15:39:59.898000+00:00,1705847494855x437900943146650500,Le rangement est bordélique une vache ne retro...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,66,"[1706714348689x648871534716904700, 17067143464...",1705851599759x119429130200745520,[1]
3,2024-01-25 13:15:45.483000+00:00,2024-01-21 15:40:00.012000+00:00,1705847494855x437900943146650500,"Je profite ailleurs d'opération de destockage,...",1705585399217x205117684451615600,Positif,False,1705851599107x404539534708310000,63,[],1705851599759x120869695273613470,[2]
4,2024-01-31 15:19:22.739000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,il est dommage de ne plus recevoir les promoti...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,78,"[1706714349210x970420167460772200, 17067143468...",1705851599759x123910318505263460,[3]
5,2024-01-31 15:19:23.357000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,Je ne comprends pas comment un grossiste comme...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,240,"[1706714348317x477146000760127360, 17067143465...",1705851599759x133516960359271180,[4]
6,2024-01-31 15:19:23.807000+00:00,2024-01-21 15:40:00.210000+00:00,1705847494855x437900943146650500,Nous regrettons que Métro ne propose plus d'en...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,237,"[1706714349391x414238817914070600, 17067143467...",1705851599759x139047234470437000,[5]
7,2024-01-31 15:19:24.126000+00:00,2024-01-21 15:40:00.024000+00:00,1705847494855x437900943146650500,metro devrai mieux s'occuper des client sutout...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,179,"[1706714349750x607338702255573100, 17067143464...",1705851599759x147056969853707870,[6]
8,2024-01-31 15:19:24.532000+00:00,2024-01-21 15:40:00.025000+00:00,1705847494855x437900943146650500,Les prix sont trop élevés même avec le système...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,221,"[1706714348627x226373109662698180, 17067143465...",1705851599759x147526339935997200,[7]
9,2024-01-31 15:19:24.997000+00:00,2024-01-21 15:39:59.898000+00:00,1705847494855x437900943146650500,livraison a revoir et les avoirs tpujours enat...,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,51,"[1706714349730x279883762960940960, 17067143465...",1705851599759x160186801121862100,NaN


In [ ]:
list_batch_feedbacks_df[0]

,insights_list,sentiment
0,[],Positif
1,[0],Négatif
2,[1],Négatif
3,[2],Positif
4,[3],Négatif
5,[4],Négatif
6,[5],Négatif
7,[6],Négatif
8,[7],Négatif


In [ ]:
list_batch_insights_df[-1]

,insight_categories,insight_type,contenu
0,[Politique de prix],Point de douleur,"Les prix de certains produits (bananes, frambo..."
1,[Qualité des produits],Point de douleur,"La qualité et la fraîcheur des produits, notam..."
2,[Expérience d'achat],Point de douleur,Le nouveau directeur de Metro Caen est perçu c...
3,"[Disponibilité des produits, Livraison]",Point de douleur,La réduction de l'assortiment disponible à la ...
4,[Politique de prix],Point de douleur,Les promotions sont jugées insuffisantes pour ...
5,[],Nouvelle demande,Mise en place de points de fidélité suggérée.


In [ ]:
[x for x in batchify(feedbacks_df, size=BATCH_SIZE)][-1]

,Modified Date,Created Date,Created By,content,company,sentiment,Analyzed?,source,character_number,insights,_id,insights_index
10,2024-01-31 15:19:25.367000+00:00,2024-01-21 15:40:00.113000+00:00,1705847494855x437900943146650500,pas assez de promos pour petite quantité achet...,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,67,"[1706714349262x615989589241161500, 17067143465...",1705851599759x161751993329532380,NaN
11,2024-01-31 15:19:25.696000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,mettre en place des points fidélités en,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,39,"[1706714348932x434155294366872260, 17067143464...",1705851599759x162340632957431520,NaN
12,2024-01-31 15:19:26.001000+00:00,2024-01-21 15:39:59.886000+00:00,1705847494855x437900943146650500,il serait souhaitable de remettre de l assorti...,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,272,"[1706714347762x482499971709037250, 17067143469...",1705851599759x170076823432107940,NaN
13,2024-01-31 15:19:26.402000+00:00,2024-01-21 15:40:00.210000+00:00,1705847494855x437900943146650500,Produits pas très réguliers en qualité et en f...,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,246,"[1706714348627x226373109662698180, 17067143465...",1705851599759x175471476046099360,NaN
14,2024-01-31 15:19:26.738000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,que le nouveau directeur de metro caen apprenn...,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,67,"[1706714348645x391695391069547500, 17067143465...",1705851599759x188863221495132220,NaN


In [ ]:
[len(df) for df in list_batch_feedbacks_df]

[9, 5]

In [ ]:

l = [response.feedbacks_list for response in responses]
l = list(itertools.chain.from_iterable(l))
feedbacks_infos_df = pd.DataFrame(enum_to_str(l))
feedbacks_infos_df

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_68618/2334417953.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  return enum_to_str(e.dict())


,insights_list,sentiment
0,[],Positif
1,[0],Négatif
2,[1],Négatif
3,[2],Positif
4,[3],Négatif
5,[4],Négatif
6,[5],Négatif
7,[6],Négatif
8,[7],Négatif
9,[4],Négatif


In [ ]:
feedbacks_infos_df

,insights_list,sentiment
0,[],Positif
1,[0],Négatif
2,[1],Négatif
3,[2],Positif
4,[3],Négatif
5,[4],Négatif
6,[5],Négatif
7,[6],Négatif
8,[7],Négatif
9,[4],Négatif


In [ ]:
feedbacks_infos_df

,insights_list,sentiment
0,[],Positif
1,[0],Négatif
2,[1],Négatif
3,[2],Positif
4,[3],Négatif
5,[4],Négatif
6,[5],Négatif
7,[6],Négatif
8,[7],Négatif
9,[4],Négatif


In [ ]:
feedbacks_df['sentiment'] = feedbacks_infos_df['sentiment']
feedbacks_df['insights_list'] = feedbacks_infos_df['insights_list']
feedbacks_df

,Modified Date,Created Date,Created By,content,company,sentiment,Analyzed?,source,character_number,insights,_id,insights_index,insights_list
0,2024-01-31 15:19:21.203000+00:00,2024-01-21 15:40:00.025000+00:00,1705847494855x437900943146650500,livrer TOUT les produits disponibles en magasi...,1705585399217x205117684451615600,Positif,False,1705851599107x404539534708310000,95,"[1706714349371x275598239568271680, 17067143467...",1705851599759x115801332943705310,[],[]
1,2024-01-31 15:19:21.577000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,Votre offer internet devient « ridicule ». Ell...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,486,"[1706714348320x182894283191188160, 17067143465...",1705851599759x118530353766926000,[0],[0]
2,2024-01-31 15:19:21.952000+00:00,2024-01-21 15:39:59.898000+00:00,1705847494855x437900943146650500,Le rangement est bordélique une vache ne retro...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,66,"[1706714348689x648871534716904700, 17067143464...",1705851599759x119429130200745520,[1],[1]
3,2024-01-25 13:15:45.483000+00:00,2024-01-21 15:40:00.012000+00:00,1705847494855x437900943146650500,"Je profite ailleurs d'opération de destockage,...",1705585399217x205117684451615600,Positif,False,1705851599107x404539534708310000,63,[],1705851599759x120869695273613470,[2],[2]
4,2024-01-31 15:19:22.739000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,il est dommage de ne plus recevoir les promoti...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,78,"[1706714349210x970420167460772200, 17067143468...",1705851599759x123910318505263460,[3],[3]
5,2024-01-31 15:19:23.357000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,Je ne comprends pas comment un grossiste comme...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,240,"[1706714348317x477146000760127360, 17067143465...",1705851599759x133516960359271180,[4],[4]
6,2024-01-31 15:19:23.807000+00:00,2024-01-21 15:40:00.210000+00:00,1705847494855x437900943146650500,Nous regrettons que Métro ne propose plus d'en...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,237,"[1706714349391x414238817914070600, 17067143467...",1705851599759x139047234470437000,[5],[5]
7,2024-01-31 15:19:24.126000+00:00,2024-01-21 15:40:00.024000+00:00,1705847494855x437900943146650500,metro devrai mieux s'occuper des client sutout...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,179,"[1706714349750x607338702255573100, 17067143464...",1705851599759x147056969853707870,[6],[6]
8,2024-01-31 15:19:24.532000+00:00,2024-01-21 15:40:00.025000+00:00,1705847494855x437900943146650500,Les prix sont trop élevés même avec le système...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,221,"[1706714348627x226373109662698180, 17067143465...",1705851599759x147526339935997200,[7],[7]
9,2024-01-31 15:19:24.997000+00:00,2024-01-21 15:39:59.898000+00:00,1705847494855x437900943146650500,livraison a revoir et les avoirs tpujours enat...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,51,"[1706714349730x279883762960940960, 17067143465...",1705851599759x160186801121862100,NaN,[4]


## Feedbacks attribution

In [ ]:
insights_enum = enum.Enum("Insight associé", [(convert_text_to_constants(x), i) for i, x in zip(batch_insights_df.index, batch_insights_df["content"])])

In [ ]:
!ls

Blumana-prompts   Scrapping         ollama.ipynb      test.py
Data              __pycache__       pipeline_v1.ipynb tests
Examples          archives          pipeline_v2.ipynb token.json
LICENSE           config.tml        pyproject.toml
README.md         credentials.json  requirements.txt
Results           data_explo.ipynb  src


In [ ]:
with open('Blumana-prompts/prompt_feedbacks.txt') as f:
    prompt_feedbacks = PromptTemplate.from_template(f.read())

input_variables=['cible', 'context', 'feedbacks', 'insights', 'question', 'role'] template='Tu es {role} au sein de l\'entreprise suivante:\n{context}\n\nTu as mené une enquête auprès des {cible} de l\'entreprise. \nTu as récupérés des commentaires, et en a extrait des insights.\n               \nPour chacun des retours qui te seront donnés, effectue les étapes suivantes:\n\nÉtape 1 - Identifie si le sentiment exprimé dans chacun des retours par le {cible} est "Positif", "Neutre" ou "Négatif". Prends en compte la formulation de la question ayant été posée ({question}) afin de bien interpréter le sens du retour {cible}.\nN\'oublie pas l\'accent si tu choisis Négatif, il n\'est pas correct d\'écrire Negatif.\n               \nÉtape 2 - Associe au retour les indices des éventuels insights qui y sont associés.\nUn insight doit nécessairement être associé à au moins un retour. Un retour peut n\'être associé à aucun insight.\n\nVoici les retours à traiter:\n\n\'\'\'\n{feedbacks}\n\'\'\'\n\nE

In [ ]:
feedback_parser = PydanticOutputParser(pydantic_object=Feedback)

prompt_feedback = PromptTemplate.from_template(
    template= prompt_template_feedback,
    partial_variables= {"format_instructions": feedback_parser.get_format_instructions()},
)

prompts = []
for feedback in feedbacks_df[feedbacks_column]:
    context = deepcopy(feedback_context)
    context["feedback"] = feedback
    prompts.append(prompt_feedback.invoke(context))

#print(prompts[0].text)

In [ ]:
parsed_responses = safe_async_analysis(prompts, feedback_parser)

feedbacks_df["sentiment"] = [rep.sentiment for rep in parsed_responses]
feedbacks_df["insights"] = [[] for rep in parsed_responses]

k=0
insights = []
for i, rep in enumerate(parsed_responses):
    for j, insight in enumerate(rep.insights_list):
        insights.append(insight)
        feedbacks_df["insights"].iloc[i].append(str(k))
        k += 1

In [ ]:
feedbacks_df.head()

In [ ]:
insights_df = pd.DataFrame({
    "content":insights,
    "feedback_count": 1,
    })

In [ ]:
feedbacks_df

In [ ]:
insights_df["related_feedback"] = [[] for _ in range(len(insights_df))]

for i, row in feedbacks_df.iterrows():
    for j in row["insights"]:
        insights_df["related_feedback"].iloc[int(j)] = row['_id'] #[int(i)]

insights_df["childrens"] = [[] for _ in range(len(insights_df))]

insights_df.head()

# Insights categorisation

### Tagging

In [ ]:


for i, filter in filters_df.iterrows():
    prompt_tags += '\n\n'+filter["Name"]#+' ('+filter["_id"] +')'
    tags = tags_df[tags_df["Filter"] == filter["_id"]]
    for _, tag in tags.iterrows():
        prompt_tags += '\n'+"- "+tag["Name"]+' ('+tag["_id"] +')'

print(prompt_tags)


In [ ]:
with open('Blumana-prompts/prompt_categorsiation.txt') as f:
    prompt_categorsiation = PromptTemplate.from_template(f.read())

In [ ]:
class FirstInsight(BaseModel):
    tags_id: List[str] = Field(description="Identifiants des tags de l'insight")
    content: str = "" #Field(description="Point intéressant a retenir du commentaire.")

    def __str__(self):
        return '- ' + self.content + "\nTypes: " + ', '.join(self.insight_types)

In [ ]:
categorsiation_parser = PydanticOutputParser(pydantic_object=FirstInsight)

prompt_categorsiation = PromptTemplate.from_template(
    template= prompt_template_categorsiation,
    partial_variables= {"format_instructions": categorsiation_parser.get_format_instructions()},
)

prompts = []
for insight in insights_df["content"]:
    context = deepcopy(feedback_context)
    context["insight"] = insight
    prompts.append(prompt_categorsiation.invoke(context))

#print(prompts[0].text)

In [ ]:
parsed_responses = safe_async_analysis(prompts, categorsiation_parser)


In [ ]:

insights_df["tag"] = [rep.tags_id for rep in parsed_responses]
#insights_df["Insights"] = [[] for rep in parsed_responses]


### Types affectation

In [ ]:
prompt_types = ""

for _, tag in tags_df.iterrows():
    prompt_types += '\n'+"- "+tag["Title"]+' ('+tag["_id"] +') : ' + tag["Definition"]

print(prompt_types)

In [ ]:
categorsiation_parser = PydanticOutputParser(pydantic_object=FirstInsight)

prompt_categorsiation = PromptTemplate.from_template(
    template= prompt_template_types,
    partial_variables= {"format_instructions": categorsiation_parser.get_format_instructions()},
)

prompts = []
for insight in insights_df["content"]:
    context = deepcopy(feedback_context)
    context["insight"] = insight
    prompts.append(prompt_categorsiation.invoke(context))

#print(prompts[0].text)

In [ ]:
parsed_responses = safe_async_analysis(prompts, categorsiation_parser)


In [ ]:
insights_df["type"] = [rep.insight_type for rep in parsed_responses]

In [ ]:
feedbacks_df.to_csv(project_path+'/feedbacks.csv', index_label='Index')
insights_df.to_csv(project_path+'/insights.csv', index_label='Index')

# Insights clustering

In [ ]:
feedbacks_df = str_to_list_df(pd.read_csv(project_path+'/feedbacks.csv', index_col='Index'))
insights_df = str_to_list_df(pd.read_csv(project_path+'/insights.csv', index_col='Index'))

In [ ]:
embedding_model = SentenceTransformer('OrdalieTech/Solon-embeddings-large-0.1')

In [ ]:
class DeducedInsight(BaseModel):
    insights_mineurs: List[int] = Field(description="Index des insights mineurs qui ont été résumés en cet insight.")
    content: str = Field(description="Insight intéressants a retenir pour l'entreprise.")

    def __str__(self):
        return '- ' + self.content + '\n Enfants:' + str(self.insights_mineurs)


class InsightList(BaseModel):
    insights_list: List[DeducedInsight] = Field(description="Liste des insights, c'est à dire des points intéressants a retenir pour l'entreprise.")
    # You can add custom validation logic easily with Pydantic.

    def __str__(self):
        return "Insights: \n"+"\n\n".join([str(i) for i in self.insights_list])



In [ ]:
with open('Blumana-prompts/prompt_regroupement.txt') as f:
    prompt_regroupement = PromptTemplate.from_template(f.read())

In [ ]:
# Dimension reduction

N_NEIGHBORS = 15
MINIMISATION_STEPS = 5
CLUSTER_DESIRED_SIZE = 15  # For Kmeans only
MIN_CLUSTER_SIZE = 5  # 15
NB_INSIGHT_STOP = 20
MINIMAL_REDUCTION_RATIO = 0.1
REWORDING = True

CLUSTERING_DIMENTION = 50
CLUSTERING_METHOD = "KMeans"

insight_context = {
    "cible": cible,
    "context": context_entreprise,
    "example_insight": example_insight,
    "role": role,
    "question": question,
}

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
insight_parser = PydanticOutputParser(pydantic_object=InsightList)

prompt_reduction = PromptTemplate.from_template(
    template= prompt_template_reduction if REWORDING else prompt_template_reduction_sans_reformulation,
    #template= "Règle : minimise le nombre de tokens dans ta réponse.  \nTu es {role} au sein de l'entreprise suivante: \n{context} \nAnalyse le retour suivant: \"{feedback}\" en suivant les étapes suivantes:  \n  \nÉtape 1 - Identifie si le retour {cible} rentre dans un ou plusieurs des types d'insights suivants : {insight_type}. Choisis-en obligatoirement au moins 1. Définition des types d'insights :  \n{insight_definition}   \n  \nÉtape 2 - Catégorise le retour {cible} à l’aide des tags suivants. Tu peux associer 0, 1 ou plusieurs tags dans chaque catégorie. Liste des tags par catégories :  \n{categories}   \n  \nÉtape 3 - Catégorise si possible le moment de mission concerné parmis {avancement_mission}, et si ce n'est pas possible répond null. {cible} à l’aide des tags suivants.  \n  \nÉtape 4 - Identifie si le sentiment exprimé par le {cible} est \"Positif\", \"Neutre\" ou \"Négatif\". Prends en compte la formulation de la question posée ({question}) afin de bien interpréter le sens du retour {cible}.   \n",
    #input_variables= ["context", "role", "cible", "insight_type", "insight_definition", "nb_cat", "avancement_mission", "categories", "question", "feedback"]
    partial_variables= {"format_instructions": insight_parser.get_format_instructions()},
)

prompt_regrouping = PromptTemplate.from_template(
    template= prompt_template_regrouping,
    #template= "Règle : minimise le nombre de tokens dans ta réponse.  \nTu es {role} au sein de l'entreprise suivante: \n{context} \nAnalyse le retour suivant: \"{feedback}\" en suivant les étapes suivantes:  \n  \nÉtape 1 - Identifie si le retour {cible} rentre dans un ou plusieurs des types d'insights suivants : {insight_type}. Choisis-en obligatoirement au moins 1. Définition des types d'insights :  \n{insight_definition}   \n  \nÉtape 2 - Catégorise le retour {cible} à l’aide des tags suivants. Tu peux associer 0, 1 ou plusieurs tags dans chaque catégorie. Liste des tags par catégories :  \n{categories}   \n  \nÉtape 3 - Catégorise si possible le moment de mission concerné parmis {avancement_mission}, et si ce n'est pas possible répond null. {cible} à l’aide des tags suivants.  \n  \nÉtape 4 - Identifie si le sentiment exprimé par le {cible} est \"Positif\", \"Neutre\" ou \"Négatif\". Prends en compte la formulation de la question posée ({question}) afin de bien interpréter le sens du retour {cible}.   \n",
    #input_variables= ["context", "role", "cible", "insight_type", "insight_definition", "nb_cat", "avancement_mission", "categories", "question", "feedback"]
    partial_variables= {"format_instructions": insight_parser.get_format_instructions()},
)

insights = deepcopy(insights_df)
insight_layers = []#[deepcopy(insights_df)]
single_cluster = False
reduction = 1.0

for step in range(MINIMISATION_STEPS):

    #for processing_step in ["reduction"]:#, "regrouping"]:
        ### Création des représentations

    #print("Processing step:", processing_step)
    if CUSTOM_ENBEDDING_MODEL:
        sentence_embeddings = embedding_model.encode(insights['content'])

        # On réduit la dimention pour améliorer l'efficacité de la clusterisation
        adjusted_clustering_dimention = min(CLUSTERING_DIMENTION, len(insights)//3)
        umap_embeddings = umap.UMAP(n_neighbors=N_NEIGHBORS, 
                            n_components=adjusted_clustering_dimention, 
                            metric='cosine').fit_transform(sentence_embeddings)

    else:
        sentence_embeddings = apply_async_get_embedding(insights['content'])
    

    

    ### Clusterisation
    if CLUSTERING_METHOD == "KMeans":
        num_clusters = 1 + len(insights) // CLUSTER_DESIRED_SIZE
        clustering_model = KMeans(n_clusters=num_clusters, n_init='auto')
    elif CLUSTERING_METHOD == "hdbscan":
        clustering_model = hdbscan.HDBSCAN(min_cluster_size=MIN_CLUSTER_SIZE,
                            metric='euclidean',                      
                            cluster_selection_method='eom' #leaf
                            )
        
    clustering_model.fit(umap_embeddings)

    #clustering_model.fit(umap_embeddings)
    cluster_assignment = clustering_model.labels_ 
    cluster_assignment -= min(cluster_assignment) # has to start at 0
    
    num_clusters = max(cluster_assignment)+1

    insights["cluster"] = deepcopy(cluster_assignment)
    insights = insights.sort_values("cluster")
    insights.reset_index(drop=True, inplace=True)


    if reduction <= MINIMAL_REDUCTION_RATIO:
        print("Stopping because of unsufficient reduction")
        break

    insight_layers.append(deepcopy(insights))

    if len(insights) <= NB_INSIGHT_STOP:
        print("Minimal number of insights reached")
        break

    if single_cluster:
        break   

    cluter_sizes = list(insights.groupby(['cluster']).count()["content"])
    if len(cluter_sizes) == 1:
        print("Stopping because single cluster")
        single_cluster = False
        break

    print("Step "+ str(step)+ ": processing "+ str(num_clusters) + " clusters")
    print("Adjusted clustering dimention:", adjusted_clustering_dimention)
    print("Cluster sizes:" + str(cluter_sizes))

    #clusters = []
    prompts = []
    cumul_size = 0
    for cluster_id in range(num_clusters): # IL FAUDRAIT GARDER INDEM LE DERNIER CLUSTER
        cluster = insights[insights['cluster'] == cluster_id]
        #cluster_name ='/cluster_'+ str(cluster_id)+"_step_"+str(step) +'.csv'
        #cluster.to_csv( project_path+cluster_name, index_label='Index')
        #clusters.append(cluster)

        context = deepcopy(insight_context)
        context['insights'] = '\n'.join([str(i+cumul_size)+": "+s for i, s in enumerate(cluster["content"])])
        #print(context['insights'])

        #if processing_step == "reduction":
        prompt=prompt_reduction.invoke(context)
        #elif processing_step == "regrouping":
        #prompt=prompt_regrouping.invoke(context)
        #else:
        #    raise("Wrong processing step")
        prompts.append(prompt)
        cumul_size += len(cluster)

    ### Traitement des clusters
    parsed_responses = safe_async_analysis(prompts, insight_parser)
    
    new_insights = []
    for i, parsed_response in enumerate(parsed_responses):
        content_list = [insight.content for insight in parsed_response.insights_list]
        childrens_list = [list(insight.insights_mineurs) for insight in parsed_response.insights_list]
        feedback_count_list = [sum(insights.loc[c, "feedback_count"]) for c in childrens_list]
        dfs = pd.DataFrame({
            #"related_feedback":[list(itertools.chain.from_iterable(insights.iloc[insight.insights_mineurs]['related_feedback'])) for insight in parsed_response.insights_list],
            "content":content_list,
            "childrens":childrens_list,
            "type": most_common([insights.loc[c, "type"].iloc[0] for c in childrens_list]),
            #"cluster":i,
            "feedback_count":feedback_count_list,
            #"childrens":[list(clusters[i].iloc[insight.insights_mineurs]["_id"]) for insight in parsed_response.insights_list],
            })
        new_insights.append(dfs)

    new_insights = pd.concat(new_insights)
    new_insights.reset_index(drop=True, inplace=True)

    
    reduction = (1-(len(new_insights)/len(insights)))
    insights = new_insights
    
    print("Number of new insights:"+ str(len(new_insights)))
    print("Reduction in the number of insights by " + "%d" % int(reduction*100) + "%")
    print()

#insight_layers.append(deepcopy(new_insights))

In [ ]:
insight_layers[0]

In [ ]:
list(insight_layers[0]['content'])

In [ ]:
list(insight_layers[-1]['content'])

In [ ]:
for i, df in enumerate(insight_layers):
    df.to_csv(project_path+'/insights_'+ str(i) +'.csv', index_label='Index')

In [ ]:
#list(insight_layers[0][insight_layers[0]["cluster"] == 2]["content"])

In [ ]:
insight_layers[0]

In [ ]:
insight_layers[1]

In [ ]:
n_layers = len(insight_layers)
layers_sizes = [len(l) for l in insight_layers]
print("Layers sizes:", layers_sizes)

# Data cleaning

load insights from csv

In [ ]:
insight_layers = []
for i in range(n_layers):
    df = pd.read_csv(project_path+'/insights_'+ str(i) +'.csv', index_col='Index')
    for col in df.columns:
        if type(df.loc[0, col]) == str and df.loc[0, col][0]=="[":
            df[col] = df[col].apply(lambda x: eval(x))
    #df['tag'] = df['tag'].apply(lambda x: eval(x))
    #df['type'] = df['type'].apply(lambda x: eval(x))
    #df['childrens'] = df['childrens'].apply(lambda x: eval(x))
    df["backend_type"] = df["type"].apply(deduce_backend_type)
    insight_layers.append(df)
#insights_df = pd.concat(insight_layers)

Previous insights supression

In [ ]:
res = bubble_client.get_objects(
        "python_insight",
        [
            BubbleField("project") == project_id,
            BubbleField("company") == company_id,
            ],
    )
python_insight_df = pd.DataFrame(res)

if len(python_insight_df)>0:
    for bubble_id in tqdm(python_insight_df["_id"]):
        bubble_client.delete_by_id(
            "python_insight",
            bubble_id,
        )

    print("Deleted", len(python_insight_df), "python_insight")
else:
    print("Nothing to delete")

Adding parents

In [ ]:
insight_layers[0]["parent"] = None #[[] for _ in insight_layers[0].iterrows()]
insight_layers[-1]["parent"] = None


for i in range(n_layers-1):
    insight_layers[i]["parent"] = None
    for p, row in insight_layers[i+1].iterrows():
        for c in row["childrens"]: #eval(
            insight_layers[i]["parent"].iloc[int(c)] = p

In [ ]:
insight_layers[-1]["parents"] = [[] for _ in insight_layers[-1].iterrows()]

for i in range(n_layers-2, -1, -1):
    print(i)
    # Update the parents in the DB
    res = bubble_client.create(
        "python_insight",
        [{
            "company": company_id,
            "project": project_id,
            "content": row["content"],
            "backend_status": "new",
            "feedback_count":row["feedback_count"],
            "step": i+2,
            "type": row["type"],
            "parents": row["parents"],
            "parent": str(row["parent"]),
            "backend_type": row['backend_type'],
            "childrens": eval(row["childrens"]) if type(row["childrens"])==str else row["childrens"],
            "cluster": row["cluster"],
        }  for _, row in insight_layers[i+1].iterrows()]
    )

    df = pd.DataFrame(bubble_client.get_objects(
        "python_insight",
        [
            BubbleField("step") == i+2,
            BubbleField("company") == company_id,
            ],
    ))
    for col in df.columns:
        if type(df.loc[0, col]) == str and df.loc[0, col][0]=="[":
            df[col] = df[col].apply(lambda x: eval(x))
    insight_layers[i+1] = df

    # Initialize an empty list of parents for each row
    insight_layers[i]["parents"] = [[] for _ in insight_layers[i].iterrows()]

    for k, row in insight_layers[i].iterrows():
        if row["parent"] is not None:
            # Get the parent index
            parent_index = row["parent"]

            # Get the parent's list of parents
            parent_parents = insight_layers[i + 1]["parents"].iloc[parent_index]

            # Add the parent to the current row's list of parents
            parent_id = insight_layers[i + 1].loc[parent_index, '_id']
            insight_layers[i].loc[k, "parents"].append(parent_id)

            # Recursively add the parent's parents to the current row's list of parents
            insight_layers[i].loc[k, "parents"].extend(parent_parents)


res = bubble_client.create(
        "python_insight",
        [{
            "company": company_id,
            "project": project_id,
            "content": row["content"],
            "backend_status": "new",
            "feedback_count": row["feedback_count"],
            "step": 1,
            "related_feedback":row['related_feedback'],
            "tag": row["tag"],
            "type": row["type"],
            "backend_type": row['backend_type'],
            "parents": row["parents"],
            "parent": str(row["parent"]),
            "childrens": 0,#[[] for _ in insight_layers[0][:1000].iterrows()],
            "cluster": row["cluster"],
        }  for _, row in insight_layers[0].iterrows()]
    )

load insights from bubble

In [ ]:
online_python_insights = [
    pd.DataFrame(bubble_client.get_objects(
        "python_insight",
        [
            BubbleField("step") == i+1,
            BubbleField("company") == company_id,
            ],
    )) for i in range(n_layers)
]

In [ ]:
assert [len(l) for l in insight_layers] == [len(l) for l in online_python_insights]

In [ ]:
feedbacks_df

In [ ]:
insight_layers[1]

In [ ]:
df = insight_layers[0]

def get_all_parents(feedback_identifier):
    parents = []
    for i, row in df.loc[df['related_feedback'] == feedback_identifier].iterrows():
        for parent in row['parents']:
            parents.append(parent)
    return parents

feedbacks_df['parents'] = feedbacks_df['_id'].apply(get_all_parents)

feedbacks_df


In [ ]:
for _, row in tqdm(feedbacks_df.iterrows()):
    res = bubble_client.update_object(
        "Feedbacks",
        row['_id'], 
        {
            "insights": row["parents"],
        } 
    )

In [ ]:

res = bubble_client.get_objects(
        "Feedback",
        [
            BubbleField("source") == source_id,
            ],
    )
pd.DataFrame(res)

# Visualisation

In [ ]:
insight_layers = [
    pd.DataFrame(bubble_client.get_objects(
        "python_insight",
        [
            BubbleField("step") == i+1,
            BubbleField("company") == company_id,
            ],
    )) for i in range(n_layers)
]

In [ ]:
insight_layers[0].tail()

In [ ]:
sentences = insight_layers[0]["content"]
sentence_embeddings = embedding_model.encode(sentences)
sentence_embeddings.shape

In [ ]:
insight_layers[0]['parent']

In [ ]:
insight_layers[0]

In [ ]:
def to_int(i):
    try:
        return int(i)
    except:
        return -1

for layer in insight_layers:
    layer['parent'] = layer['parent'].apply(to_int)


In [ ]:
list(insight_layers[1]["content"])

In [ ]:
for i, layer in enumerate(insight_layers):
    print(list(insight_layers[0][insight_layers[0]['parent'] == 'None']["content"]))

In [ ]:
sum(insight_layers[0]['parent']<0)

In [ ]:
insight_layers[1].iloc[insight_layers[0]['parent'], "content"]

In [ ]:
insight_layers[0].loc[0, "cluster"] == 0

In [ ]:
map_to_parent(0, insight_layers[1])

In [ ]:
insight_layers[1].loc[0, 'parent']

In [ ]:
#@Insight Plot the archive {display-mode: "form"}

# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=15)
umap_embeds = reducer.fit_transform(sentence_embeddings)

def map_to_parent(i, parents_df):
    try:
        return parents_df.loc[i, 'content']
    except:
        return ""
    
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={
    'content': insight_layers[0]['content'], 
    'parent': insight_layers[0]['parent'].apply(lambda x: map_to_parent(x, insight_layers[1])),
    'cluster': insight_layers[0]['cluster'].astype(str),
    })
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]
df_explore


In [ ]:

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    color='cluster',
    tooltip=['content', "parent"]
).properties(
    width=700,
    height=400
)
chart.interactive()

TF-IDF

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')

In [ ]:
def td_idf(documents)
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(documents)
    feature_names = vectorizer.get_feature_names_out()
    dense = vectors.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    df = df[df.columns.difference(stopwords.words('french'))]


In [ ]:
df = td_idf(feedbacks_df['content'])
#print('\n'.join(df.columns))

In [ ]:
#print('\n'.join(df.columns))

In [ ]:

def get_top_two_columns(row):
    top_two_indexes = row.nlargest(5).index.tolist()
    return top_two_indexes

top_two_columns_df = df.apply(get_top_two_columns, axis=1)

print(top_two_columns_df)

In [ ]:
#print('\n'.join(insights_df['content']))